# Tehtävä 1
### Aihe: Keras Functional API:n käyttö
2,5 pistettä

1. Luo alla olevan kuvan mukainen neuroverkko, jossa on useampi sisääntulo.
    * Kerroksien neuronien määrällä tai aktivaatiofunktiolla ei ole väliä, määritä ne itse vapaasti.
2. Aja "Tehtävän vastaukset" solu

Vinkkejä:
* Käytä concatenate - kerrosta yhdistääksesi Input2 ja sitä edeltävän Densen ulostulo. Sama homma Input3:lle. Katso esimerkki materiaalista Keras - otsikon alta.
* Add kerroksen dokumentaatio: https://www.tensorflow.org/api_docs/python/tf/keras/layers/add?hl=th

In [ ]:
from IPython.display import Image
Image('../img/keras_functional_tehtävä.png')

In [1]:
# Tehtävän toteutus
import tensorflow as tf
tf.keras.backend.set_floatx('float64')

boston = tf.keras.datasets.boston_housing
(train_features, train_labels), (test_features, test_labels) = boston.load_data()

train_features_input1, train_features_input2 = train_features[:,:2], train_features[:,2]
print(train_features_input1.shape)
print(train_features_input2.shape)

#inputs
input1 = tf.keras.Input(shape=(13))
input2 = tf.keras.Input(shape=(13))
input3 = tf.keras.Input(shape=(13))

#initial dense layers
dense1 = tf.keras.layers.Dense(13, activation='relu')(input1)
dense2 = tf.keras.layers.Dense(26, activation='relu')(dense1)
dense3 = tf.keras.layers.Dense(20, activation='relu')(dense2)

#concat dense3 to input2 and input3 layer
concat1 = tf.keras.layers.concatenate([input2, dense3])
concat2 = tf.keras.layers.concatenate([input3, dense3])

#left side after concats
dense4 = tf.keras.layers.Dense(23, activation='relu')(concat1)
dropout1 = tf.keras.layers.Dropout(.20)(dense4)
dense5 = tf.keras.layers.Dense(46, activation='relu')(dropout1)

#right side after concats
dense6 = tf.keras.layers.Dense(23, activation='relu')(concat2)
dropout2 = tf.keras.layers.Dropout(.30)(dense6)
dense7 = tf.keras.layers.Dense(46, activation='relu')(dropout2)

#final 3 layers
add1 = tf.keras.layers.add([dense5, dense7])
dense8 = tf.keras.layers.Dense(20, activation='relu')(add1)
dense9 = tf.keras.layers.Dense(10, activation='relu')(dense8)

model_functional = tf.keras.Model(inputs=(input1, input2, input3),
                                 outputs=dense9)


(404, 2)
(404,)


In [2]:
# Tehtävän vastaukset. Huom! Älä muokkaa tätä solua, vaan aja se, kun olet suorittanut tehtävän. Sijoita luomasi malli model_functional - muuttujaan.
model_functional.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 13)]         0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 13)           182         input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 26)           364         dense[0][0]                      
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 13)]         0                                            
_______________________________________________________________________________________

# Tehtävä 2
### Aihe: Konvoluutioneuroverkko ja feature - matriisit
2,5 pistettä

1. Lataa MNIST datasetti.
2. Luo konvoluutioneuroverkko, joka vie klassifiointikerroksille 32 feature - matriisia, joiden korkeus ja leveys on 6.
3. Todista neuroverkon toimivuus kouluttamalla MNIST datasettiä muutama epookki.
4. Aja "Tehtävän vastaukset" solu

Vinkkejä:
* Käytä useita konvoluutio ja Max Pooling - kerroksia saavuttaaksesi oikean korkeuden ja leveyden feature - matriiseille. Käytä apuna ulostulon laskukaavoja materiaaleista.
    * Mahdollisia yhdistelmiä on monta, ei yhtä ainoaa ratkaisua. 
* Jos koneellasi loppuu muisti kesken tai konvoluutioverkon koulutus kestää liian kauan, pienennä MNIST datasetin kokoa esim. kymmenesosaan.
* Klassifikaatiokerrokset = Flatten, Dense kerrokset konvoluutiokerrosten jälkeen.

In [3]:
# Tehtävän toteutus
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

(train_X, train_y), (test_X, test_y) = tf.keras.datasets.mnist.load_data()
print(train_X.shape)

#plt.imshow(x_train[0], cmap='Greys') #black and white

#input variables
train_X = train_X.reshape((train_X.shape[0],28,28,1))
test_X = test_X.reshape((test_X.shape[0],28,28,1))
print(test_X.shape)
train_X = train_X/255 # scaled to values in between 0-1
test_X = test_X/255

train_y = pd.get_dummies(train_y)
test_y = pd.get_dummies(test_y)

# Jos padding on 'same', niin matriisia, johon konvoluutio suoritetaan, täytetään nollilla, niin että ulostulossa on samat dimensiot (leveys ja korkeus) kuin sisääntulossa.
# Tämä pätee vain, jos askel on yksi
model_input = tf.keras.Input(shape=(28,28,1)) 

# konvoluutiokerros
model_conv1 = tf.keras.layers.Conv2D(filters=32, kernel_size=(2,2),strides=1, padding='same')(model_input) #28x28x32
model_conv2 = tf.keras.layers.Conv2D(filters=32, kernel_size=(2,2),strides=(2,2), padding='same')(model_conv1)
model_maxpool1 = tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))(model_conv2)
model_maxpool2 = tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(1,1))(model_maxpool1) #6x6x32

#Flatten
model_flatten = tf.keras.layers.Flatten()(model_maxpool2)

#Dense
model_dense1 = tf.keras.layers.Dense(50, activation='relu')(model_flatten)
model_dense2 = tf.keras.layers.Dense(40, activation='relu')(model_dense1)
model_dense3 = tf.keras.layers.Dense(32, activation='relu')(model_dense2)
model_dense4 = tf.keras.layers.Dense(26, activation='relu')(model_dense3)
model_dense5 = tf.keras.layers.Dense(24, activation='relu')(model_dense4)

output_layer = tf.keras.layers.Dense(10, activation='softmax')(model_dense5)

model_mnist = tf.keras.Model(inputs=model_input,
                        outputs=output_layer)
print(model_mnist.layers[4].output)
print(model_mnist.output)

model_mnist.compile(loss=['categorical_crossentropy'],
               optimizer=tf.optimizers.Adam(learning_rate=0.001),
               metrics=['categorical_accuracy'])

model_mnist.fit(train_X, train_y, validation_data=(test_X, test_y), epochs=2, batch_size=100)

(60000, 28, 28)
(10000, 28, 28, 1)
Tensor("max_pooling2d_1/MaxPool:0", shape=(None, 6, 6, 32), dtype=float64)
Tensor("dense_14/Softmax:0", shape=(None, 10), dtype=float64)
Epoch 1/2
600/600 [==============================] - 23s 38ms/step - loss: 0.5650 - categorical_accuracy: 0.8114 - val_loss: 0.1508 - val_categorical_accuracy: 0.9552
Epoch 2/2
600/600 [==============================] - 23s 38ms/step - loss: 0.1321 - categorical_accuracy: 0.9589 - val_loss: 0.0931 - val_categorical_accuracy: 0.9714


In [ ]:
# Tehtävän vastaukset. Huom! Älä muokkaa tätä solua, vaan aja se, kun olet suorittanut tehtävän. 

# Sijoita layer_output muuttujaan sen konvoluutikerroksen ulostulon muoto, missä kerroksesta tulee ulos 32 feature matriisia, joiden korkeus ja leveys on 6
# Esim. layer_output = model.layers[5].output.shape
print(output_layer)
# Sijoita luomasi malli model_mnist - muuttujaan.
model_mnist.summary()

# Tehtävä 3
### Aihe: Siirto-oppiminen
2,5 pistettä

1. Luo alla olevan kuvan mukainen neuroverkkomalli.
2. Lataa malliin painoarvot "weights.h5" tiedostosta.
3. Aja tehtävän viimeinen "Vastaukset" solu.

In [ ]:
Image('../img/cnnmalli.png')

Vinkkejä:
* Käytä Conv2D - kerroksissa aktivaatiofunktiona 'relu'
* Katso konvoluutiokerroksen ulostulosta, kuinka monta filtteriä tulisi määrittää.
* Konvoluutiokerroksissa kernelin koko on (5,5). Katso materiaaleista, mikä padding ja stride tulee olla, että ulostulolla on sama leveys ja korkeus kuin sisääntulolla.
* MaxPooling2D:ssä pool_size on (2,2). Mikä tulee olla askel (strides) parametri, että sisääntulo puolittuu?
* Dropout deaktivoi ensimmäisessä Dropout - kerroksessa 20% neuroneista ja toisessa 40%.

In [1]:
# Tehtävän toteutus
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

(train_X, train_y), (test_X, test_y) = tf.keras.datasets.mnist.load_data()
#print(train_X.shape)

train_X = train_X.reshape((train_X.shape[0],28,28,1))
#print(train_X.shape)

model_input = tf.keras.Input(shape=(28,28,1))
model_conv1 = tf.keras.layers.Conv2D(filters=24, kernel_size=(5,5),strides=1, padding='same', activation='relu')(model_input)
model_maxpool1 = tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2))(model_conv1)
model_dropout1 = tf.keras.layers.Dropout(.20)(model_maxpool1)
model_conv2 = tf.keras.layers.Conv2D(filters=48, kernel_size=(5,5),strides=1, padding='same', activation='relu')(model_dropout1)
model_dropout2 = tf.keras.layers.Dropout(.40)(model_conv2)
model_conv3 = tf.keras.layers.Conv2D(filters=64, kernel_size=(5,5),strides=1, padding='same', activation='relu')(model_dropout2)
model_maxpool2 = tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2))(model_conv3)
model_flatten = tf.keras.layers.Flatten()(model_maxpool2)
model_dense1 = tf.keras.layers.Dense(256, activation='relu')(model_flatten)
model_dense2 = tf.keras.layers.Dense(10, activation='relu')(model_dense1)

model_cnn = tf.keras.Model(inputs=model_input,
                        outputs=model_dense2)

model_cnn.load_weights('osa2_weights.h5')

#model_cnn.summary()

In [4]:
# Tehtävän vastaukset. Huom! Älä muokkaa tätä solua, vaan aja se, kun olet suorittanut tehtävän. Sijoita luomasi malli model_cnn - muuttujaan
model_cnn.summary()

NameError: name 'model_cnn' is not defined

# Tehtävä 4
### Aihe: Siirto-oppiminen
2,5 pistettä
 
1. Lataa Fashion MNIST datasetti ajamalla tehtävän ensimmäinen solu.
2. Poista äskeisessä tehtävässä luodusta mallista klassifikaatiokerrokset. (eli Flatten ja kaikki sen jälkeiset kerrokset)
3. Jäädytä loput kerrokset.
4. Luo klassifikaatiokerrokset ja lisää ne malliin.
5. Kouluta mallia Fashion MNIST datasetillä muutama kierros (epoch) käyttäen train_X ja train_y koulutusdataa.
6. Aja tehtävän viimeinen "Vastaukset" solu. 

Vinkkejä:
* Fashion MNIST on MNISTin tapainen datasetti
    * Muokkaa Fashion MNIST datasetin kuvien muoto, jotta ne voi syöttää neuroverkolle.
* Käy neuroverkon kerrokset läpi ja muuta kerrosten 'trainable' parametriä. Näin voit jäädyttää kerroksen.
* Klassifikaatiokerrokset = Flatten, Dense kerrokset.

In [4]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

(train_X, train_y), (test_X, test_y) = tf.keras.datasets.fashion_mnist.load_data()
# Tehtävän toteutus tähän

#input variables
train_X = train_X.reshape((train_X.shape[0],28,28,1))
test_X = test_X.reshape((test_X.shape[0],28,28,1))
train_X = train_X/255 # scaled to values in between 0-1
test_X = test_X/255

#OneHot transformation
train_y = pd.get_dummies(train_y)
test_y = pd.get_dummies(test_y)

#print(test_y.shape)

model_freeze = model_cnn.layers[:8]

for layer in model_freeze:
    #print(layer)
    layer.trainable = False


#classification layers
#print(model_freeze[-1])

new_model_flatten = tf.keras.layers.Flatten()(model_freeze[-1].output)
new_model_dense1 = tf.keras.layers.Dense(256, activation='relu')(new_model_flatten)

#output layer
new_output_layer = tf.keras.layers.Dense(10 ,activation='softmax')(new_model_dense1)

model_result = tf.keras.Model(inputs=model_freeze[0].input,
                              outputs=new_output_layer)

model_result.summary()


model_result.compile(loss=['categorical_crossentropy'],
                     optimizer=tf.optimizers.Adam(learning_rate=0.001),
                     metrics=['categorical_accuracy'])


model_result.fit(train_X,
                 train_y,
                 epochs=10)


results = model_result.evaluate(train_X,
                                train_y,
                                verbose=0)

#results.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 24)        624       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 24)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 24)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 48)        28848     
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 48)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)       

In [5]:
# Tehtävän vastaukset. Huom! Älä muokkaa tätä solua, vaan aja se, kun olet suorittanut tehtävän. Sijoita results - muuttujaan funktion model.evaluate() tulos.
# Muista määrittää model.compile() - funktioon seurattavaksi suureeksi metrics=['accuracy'], jotta näät, kuinka suuri osa neuroverkon ennustuksista on oikein.
print(f"Test Loss:{results[0]} Test Accuracy:{results[1]*100}%")

Test Loss:0.18510577082633972 Test Accuracy:93.28166842460632%
